In [144]:
from __future__ import division
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score

In [145]:
train = pd.read_csv('../data/input/train.csv')
enbase = pd.read_csv('../data/input/1entbase.csv')
alter = pd.read_csv('../data/input/2alter.csv')
branch = pd.read_csv('../data/input/3branch.csv')
invest = pd.read_csv('../data/input/4invest.csv')
right = pd.read_csv('../data/input/5right.csv')
project = pd.read_csv('../data/input/6project.csv')
lawsuit = pd.read_csv('../data/input/7lawsuit.csv')
breakfaith = pd.read_csv('../data/input/8breakfaith.csv')
recruit = pd.read_csv('../data/input/9recruit.csv')
qualification = pd.read_csv('../data/input/10qualification.csv')
test = pd.read_csv('../data/input/evaluation_public.csv')

In [146]:
enbase = enbase.fillna(value={'ZCZB': 0, 'MPNUM': 0, 'INUM': 0, 'ENUM': 0, 'FINZB': 0, 'FSTINUM': 0, 'TZINUM': 0})  # 未处理 HY；ZCZB 为 0 表示缺失或错误

In [147]:
enbase.shape

(436798, 12)

In [148]:
def translate_date(date):
    year = int(date[:4])
    month = int(date[-2:])
    return (year - 2010) * 12 + month

In [149]:
def get_alter_feature(df):
    df = df.copy()

    alt_no = df.groupby(['EID', 'ALTERNO']).size().reset_index()
    alt_no = alt_no.groupby('EID')[0].agg([sum, len]).reset_index()
    alt_no.columns = ['EID', 'alt_count', 'alt_types_count']

    alt_no_oh = df.groupby(['EID', 'ALTERNO']).size().unstack().reset_index()
    alt_no_oh.columns = [i if i == 'EID' else 'alt_' + i for i in alt_no_oh.columns]

    df['date'] = df['ALTDATE'].apply(translate_date)
    date = df.groupby('EID')['date'].agg([min, max, np.ptp, np.std]).reset_index()
    date.columns = ['EID', 'alt_date_min', 'alt_date_max', 'alt_date_ptp', 'alt_date_std']

    df['altbe'] = df['ALTBE'].str.extract('(\d+\.?\d*)').astype(float)
    df['altaf'] = df['ALTAF'].str.extract('(\d+\.?\d*)').astype(float)
    alt_be_af = df.groupby('EID')['altbe', 'altaf'].agg([min, max, np.mean]).reset_index()
    alt_be_af.columns = ['EID', 'alt_be_min', 'alt_be_max', 'alt_be_mean', 'alt_af_min', 'alt_af_max', 'alt_af_mean']

    mydf = pd.merge(alt_no, alt_no_oh, how='left', on='EID')
    mydf = pd.merge(mydf, date, how='left', on='EID')
    mydf = pd.merge(mydf, alt_be_af, how='left', on='EID')

    return mydf

In [150]:
def get_right_feature(df):
    df = df.copy()
    
    rig_type = df.groupby(['EID', 'RIGHTTYPE']).size().reset_index()
    rig_type = rig_type.groupby('EID')[0].agg([sum, len]).reset_index()
    rig_type.columns = ['EID', 'rig_count', 'rig_types_count']
    
    rig_type_oh_rate = df.groupby(['EID', 'RIGHTTYPE']).size().unstack().reset_index()
    rig_type_oh_rate.iloc[:, 1:] = rig_type_oh_rate.iloc[:, 1:].div(rig_type['rig_count'], axis='index')
    rig_type_oh_rate.columns = [i if i == 'EID' else 'rig_rate_' + str(i) for i in rig_type_oh_rate.columns]
    
    df['ask_month'] = (pd.to_datetime(df['ASKDATE']).dt.to_period("M") - (pd.to_datetime('2010-01-01').to_period("M"))).fillna(-999).astype(int).replace(-999, np.NaN)
    ask_date = df.groupby('EID')['ask_month'].agg([max, min, np.ptp, np.std]).reset_index()
    ask_date.columns = ['EID', 'rig_askdate_max', 'rig_askdate_min', 'rig_askdate_ptp', 'rig_askdate_std']

    df['get_month'] = (pd.to_datetime(df['FBDATE']).dt.to_period("M") - (pd.to_datetime('2010-01-01').to_period("M"))).fillna(-999).astype(int).replace(-999, np.NaN)
    get_date = df.groupby('EID')['get_month'].agg([max, min, np.ptp, np.std]).reset_index()
    get_date.columns = ['EID', 'rig_getdate_max', 'rig_getdate_min', 'rig_getdate_ptp', 'rig_getdate_std']
    
    # bad
    unget = df[df.FBDATE.isnull()]
    unget = unget.groupby('EID').size().reset_index()
    unget.columns = ['EID', 'rig_unget_num']
    
    right_1year = df[df['ASKDATE'] >= '2015-01'].groupby('EID')['ASKDATE'].count().reset_index()
    right_1year.columns = ['EID', 'ask_num(1year)']
    right_2year = df[df['ASKDATE'] >= '2014-01'].groupby('EID')['ASKDATE'].count().reset_index()
    right_2year.columns = ['EID', 'ask_num(2year)']
    right_5year = df[df['ASKDATE'] >= '2010-01'].groupby('EID')['ASKDATE'].count().reset_index()
    right_5year.columns = ['EID', 'ask_num(5year)']
    right_end_1year = df[df['FBDATE'] >= '2015-01'].groupby('EID')['FBDATE'].count().reset_index()
    right_end_1year.columns = ['EID', 'get_num(1year)']
    right_end_2year = df[df['FBDATE'] >= '2014-01'].groupby('EID')['FBDATE'].count().reset_index()
    right_end_2year.columns = ['EID', 'get_num(2year)']
    right_end_5year = df[df['FBDATE'] >= '2010-01'].groupby('EID')['FBDATE'].count().reset_index()
    right_end_5year.columns = ['EID', 'get_num(5year)']
    
    mydf = pd.merge(rig_type, rig_type_oh_rate, how='left', on='EID')
    mydf = pd.merge(mydf, ask_date, how='left', on='EID')
    mydf = pd.merge(mydf, get_date, how='left', on='EID')
    mydf = pd.merge(mydf, unget, how='left', on='EID')
    mydf = pd.merge(mydf, right_1year, how='left', on='EID')
    mydf = pd.merge(mydf, right_2year, how='left', on='EID')
    mydf = pd.merge(mydf, right_5year, how='left', on='EID')
    mydf = pd.merge(mydf, right_end_1year, how='left', on='EID')
    mydf = pd.merge(mydf, right_end_2year, how='left', on='EID')
    mydf = pd.merge(mydf, right_end_5year, how='left', on='EID')
    
    # bad
    mydf['ask_rate(1year)'] = mydf['ask_num(1year)'] / mydf['rig_count']
    mydf['ask_rate(2year)'] = mydf['ask_num(2year)'] / mydf['rig_count']
    mydf['ask_rate(5year)'] = mydf['ask_num(5year)'] / mydf['rig_count']
    mydf['get_rate(1year)'] = mydf['get_num(1year)'] / mydf['rig_count']
    mydf['get_rate(2year)'] = mydf['get_num(2year)'] / mydf['rig_count']
    mydf['get_rate(5year)'] = mydf['get_num(5year)'] / mydf['rig_count']

    return mydf

In [151]:
def get_recruit_feature(df):
    df = df.copy()
    
    rec_wz = df.groupby(['EID', 'WZCODE']).size().reset_index()
    rec_wz = rec_wz.groupby('EID')[0].agg([sum, len]).reset_index()
    rec_wz.columns = ['EID', 'rec_wz_count', 'rec_wz_types_count']
    
    # bad
    rec_wz_oh = df.groupby(['EID', 'WZCODE']).size().unstack().reset_index()
    rec_wz_oh.columns = [i if i == 'EID' else 'rec_wz_' + i for i in rec_wz_oh.columns]
    
    # bad
    rec_pos = df.groupby(['EID', 'POSCODE']).size().reset_index()
    rec_pos = rec_pos.groupby('EID')[0].agg([sum, len]).reset_index()
    rec_pos.columns = ['EID', 'rec_pos_count', 'rec_pos_types_count']
    
    df['recdate'] = (pd.to_datetime(df['RECDATE']).dt.to_period("M") - (pd.to_datetime('2010-01-01').to_period("M"))).fillna(-999).astype(int).replace(-999, np.NaN)
    rec_date = df.groupby('EID')['recdate'].agg([max, min, np.ptp, np.std]).reset_index()
    rec_date.columns = ['EID', 'rec_date_max', 'rec_date_min', 'rec_date_ptp', 'rec_date_std']
    
    # bad
    df['pnum'] = df['PNUM'].str.extract('(\d+)').fillna(1).astype(int)  # 若干=1
    rec_num = df.groupby('EID')['pnum'].agg([sum, max, min, np.ptp, np.std]).reset_index()
    rec_num.columns = ['EID' if i == 'EID' else 'rec_num_' + i for i in rec_num.columns]
    
    mydf = pd.merge(rec_wz, rec_wz_oh, how='left', on='EID')
    mydf = pd.merge(mydf, rec_pos, how='left', on='EID')
    mydf = pd.merge(mydf, rec_date, how='left', on='EID')
    mydf = pd.merge(mydf, rec_num, how='left', on='EID')

    return mydf

In [183]:
df = branch.copy()

bra_num = df.groupby('EID')['TYPECODE'].size().reset_index()
bra_num.columns = ['EID', 'bra_count']

bra_home = df.groupby(['EID', 'IFHOME']).size().unstack().reset_index()
bra_home.columns = ['EID', 'bra_nothome', 'bra_home']

bra_year = df.groupby('EID')['B_REYEAR'].agg([min, max, np.ptp, np.std]).reset_index()
bra_year.columns = [i if i == 'EID' else 'bra_year_' + i for i in bra_year.columns]

In [185]:
def get_branch_feature(df):
    df = df.copy()
    
    bra_num = df.groupby('EID')['TYPECODE'].size().reset_index()
    bra_num.columns = ['EID', 'bra_count']
    
    # bad
    bra_home = df.groupby(['EID', 'IFHOME']).size().unstack().reset_index()
    bra_home.columns = ['EID', 'bra_nothome', 'bra_home']
    
    bra_year = df.groupby('EID')['B_REYEAR'].agg([min, max, np.ptp, np.std]).reset_index()
    bra_year.columns = [i if i == 'EID' else 'bra_year_' + i for i in bra_year.columns]
    
    mydf = pd.merge(bra_num, bra_home, how='left', on='EID')
    mydf = pd.merge(mydf, bra_year, how='left', on='EID')

    return mydf

In [154]:
alter_feat = get_alter_feature(alter)

F:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app
F:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


In [155]:
right_feature = get_right_feature(right)

In [156]:
recruit_feat = get_recruit_feature(recruit)

F:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [186]:
branch_feat = get_branch_feature(branch)

In [187]:
dataset = pd.merge(enbase, alter_feat, on='EID', how='left')
dataset = pd.merge(dataset, right_feature, on='EID', how='left')
dataset = pd.merge(dataset, recruit_feat, on='EID', how='left')
dataset = pd.merge(dataset, branch_feat, on='EID', how='left')

In [188]:
trainset = pd.merge(train, dataset, on='EID', how='left')
testset = pd.merge(test, dataset, on='EID', how='left')

In [189]:
# EID 前面的字母代表不同省份，已提供了 PROV 列，因此字母是冗余信息，直接舍弃
trainset['EID'] = trainset['EID'].str.extract('(\d+)').astype(int)
testset['EID'] = testset['EID'].str.extract('(\d+)').astype(int)

F:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
F:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [190]:
train_feature = trainset.drop(['TARGET', 'ENDDATE'], axis=1)
train_label = trainset.TARGET.values
test_feature = testset
test_index = testset.EID.values
print train_feature.shape, train_label.shape, test_feature.shape

(218264, 89) (218264L,) (218247, 89)


In [191]:
config = {
    'rounds': 10000,
    'folds': 3
}

params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
#     'objective': 'rank:pairwise',
    'stratified': True,
    'scale_pos_weights ': 0,
    'max_depth': 6,
    'min_child_weight': 1,
    'gamma': 1,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'lambda': 1,

    'eta': 0.02,
    'seed': 20,
    'silent': 1,
    'eval_metric': 'auc'
}

In [192]:
def xgb_cv(train_feature, train_label, params, rounds):
    print train_feature.columns
    params['scale_pos_weights '] = float(len(train_label[train_label == 0])) / len(train_label[train_label == 1])
    dtrain = xgb.DMatrix(train_feature, label=train_label)
    num_round = rounds
    print 'run cv: ' + 'round: ' + str(rounds)
    res = xgb.cv(params, dtrain, num_round, verbose_eval=10, early_stopping_rounds=100)
    return len(res)


def xgb_predict(train_feature, train_label, test_feature, rounds, params):
    params['scale_pos_weights '] = float(len(train_label[train_label == 0])) / len(train_label[train_label == 1])
    dtrain = xgb.DMatrix(train_feature, label=train_label)
    dtest = xgb.DMatrix(test_feature, label=np.zeros(test_feature.shape[0]))
    watchlist = [(dtrain, 'train')]
    num_round = rounds
    model = xgb.train(params, dtrain, num_round, watchlist, verbose_eval=50)
    predict = model.predict(dtest)
    return model, predict


def store_result(test_index, pred, threshold, name):
    result = pd.DataFrame({'EID': test_index, 'FORTARGET': 0, 'PROB': pred})
    mask = result['PROB'] >= threshold
    result.at[mask, 'FORTARGET'] = 1
    # result['PROB'] = result['PROB'].apply(lambda x: round(x, 4))
    result.to_csv('../data/output/sub/' + name + '.csv', index=0)
    return result

In [193]:
iterations = xgb_cv(train_feature, train_label, params, config['rounds'])

Index([u'EID', u'PROV', u'RGYEAR', u'HY', u'ZCZB', u'ETYPE', u'MPNUM', u'INUM',
       u'ENUM', u'FINZB', u'FSTINUM', u'TZINUM', u'alt_count',
       u'alt_types_count', u'alt_01', u'alt_02', u'alt_03', u'alt_04',
       u'alt_05', u'alt_10', u'alt_12', u'alt_13', u'alt_14', u'alt_27',
       u'alt_99', u'alt_A_015', u'alt_date_min', u'alt_date_max',
       u'alt_date_ptp', u'alt_date_std', u'alt_be_min', u'alt_be_max',
       u'alt_be_mean', u'alt_af_min', u'alt_af_max', u'alt_af_mean',
       u'rig_count', u'rig_types_count', u'rig_rate_11', u'rig_rate_12',
       u'rig_rate_20', u'rig_rate_30', u'rig_rate_40', u'rig_rate_50',
       u'rig_rate_60', u'rig_askdate_max', u'rig_askdate_min',
       u'rig_askdate_ptp', u'rig_askdate_std', u'rig_getdate_max',
       u'rig_getdate_min', u'rig_getdate_ptp', u'rig_getdate_std',
       u'rig_unget_num', u'ask_num(1year)', u'ask_num(2year)',
       u'ask_num(5year)', u'get_num(1year)', u'get_num(2year)',
       u'get_num(5year)', u'ask_rate(1y

Will train until cv error hasn't decreased in 100 rounds.
[0]	cv-test-auc:0.632560333333+0.00189615072069	cv-train-auc:0.639232+0.00284711795798
[10]	cv-test-auc:0.65017+0.00768383354843	cv-train-auc:0.663998333333+0.00214365207583
[20]	cv-test-auc:0.655122666667+0.00390477804178	cv-train-auc:0.669553333333+0.00171023825501
[30]	cv-test-auc:0.657412666667+0.00357186639666	cv-train-auc:0.672686666667+0.00153855913836
[40]	cv-test-auc:0.659599666667+0.00281722728149	cv-train-auc:0.675220333333+0.00187440023
[50]	cv-test-auc:0.660518333333+0.00272018287784	cv-train-auc:0.676879666667+0.0015326735537
[60]	cv-test-auc:0.661703+0.00254826856251	cv-train-auc:0.678265+0.00151961047641
[70]	cv-test-auc:0.662205+0.00280434389237	cv-train-auc:0.679908666667+0.00138798398966
[80]	cv-test-auc:0.662939+0.0027550616448	cv-train-auc:0.681307333333+0.00129745503026
[90]	cv-test-auc:0.664005333333+0.00287497945886	cv-train-auc:0.682670333333+0.00142480977288
[100]	cv-test-auc:0.664923+0.00282428055736	c

[910]	cv-test-auc:0.676268666667+0.00351466416919	cv-train-auc:0.750532333333+0.00107720172463
[920]	cv-test-auc:0.676273666667+0.00355066100262	cv-train-auc:0.751126666667+0.00104973086498
[930]	cv-test-auc:0.676284+0.00354517446679	cv-train-auc:0.751747333333+0.00101797358621
[940]	cv-test-auc:0.676273333333+0.00355242690127	cv-train-auc:0.752328+0.00106354783625
[950]	cv-test-auc:0.676287+0.00353144993829	cv-train-auc:0.753025+0.000989739696419
[960]	cv-test-auc:0.676283333333+0.00354491996462	cv-train-auc:0.75353+0.00106567255759
[970]	cv-test-auc:0.676319+0.00353914857934	cv-train-auc:0.754089666667+0.00104804845096
[980]	cv-test-auc:0.676292666667+0.00353182109148	cv-train-auc:0.754643666667+0.00103756167153
[990]	cv-test-auc:0.676329+0.00353135479196	cv-train-auc:0.755249333333+0.000989542093878
[1000]	cv-test-auc:0.676340666667+0.00352567774414	cv-train-auc:0.755719333333+0.000950256222757
[1010]	cv-test-auc:0.676364666667+0.0035370669896	cv-train-auc:0.756146333333+0.000961090

In [194]:
import winsound
winsound.Beep(600,1000)

In [ ]:
model, pred = xgb_predict(train_feature, train_label, test_feature, iterations, params)

In [ ]:
importance = pd.DataFrame(model.get_fscore().items(), columns=['feature','importance']).sort_values('importance', ascending=False)
importance.to_csv('../data/output/feat_imp/importance.csv', index = False)

In [ ]:
res = store_result(test_index, pred, 0.5, '1122-xgb-entbase_only.csv')